In [1]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
from collections import Counter
from tqdm import tqdm
diretorio = r'Bases/Arquivos_Limpos/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
TESTE = pd.read_csv(f'{diretorio}{all_files[0]}', names=['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV'], sep=';', encoding='utf-8',usecols=[0,1,2], low_memory=False) 


In [4]:
CNPJ_COMPLETO = TESTE['CNPJ_BASE'].map(str) + TESTE['CNPJ_ORDEM'].map(str) + TESTE['CNPJ_DV'].map(str)

In [5]:
TESTE['CNPJ_DV'].nunique()

100